# Bayesian AGN Decomposition Analysis for SDSS Spectra (Python 2, version 5.0) 

### by Remington O. Sexton 

some would say its pretty BADASS...

In [ ]:
# % matplotlib notebook
# % matplotlib inline
import glob
from time import clock
from os import path
import os
import shutil
from astropy.io import fits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from numpy.polynomial import legendre, hermite
from scipy import linalg, special, fftpack
import sys
import emcee
import matplotlib.gridspec as gridspec
from numpy import linspace, meshgrid
from scipy.interpolate import griddata, interp1d
from matplotlib import cm
import psutil
import time
import matplotlib.gridspec as gridspec
from scipy.integrate import simps
import datetime
from astropy.stats import mad_std
import natsort
import matplotlib
matplotlib.rcParams['agg.path.chunksize'] = 100000
import badass_v5 as badass

In [ ]:
############################## Directory Structure ##########################################################
spec_dir = 'example_spectra/'
# spec_dir = 'ppxf_example/'

ppxf_dir = 'badass_data_files/'
temp_dir = ppxf_dir+'indo_us_library'
# Get full list of spectrum folders; these will be the working directories
spec_loc = natsort.natsorted( glob.glob(spec_dir+'J*') )
# ###########################################################################################################

# Loop over all objects we want to fit in spec_loc
# for i in range(0,len(spec_loc),1):
for i in range(0,1,1):
    # We need to reset options to default for each object:
    ################################### Options ################################################################
    # Fitting parameters
    fit_reg = (4400,5800) # Fitting region; MILES Stellar Library=(3540,7409); Indo-US Library=(3460,9464)
    # fit_reg = 'auto' # Automatically choose maximum fitting region based on restframe spectrum 
    good_thresh = 0.60 # good pixels threshold; if number of good pixels is below this percentage, do NOT fit the spectrum.
    mcbs_niter = 50 # number of monte carlo bootstrap simulations for outflows
    # MCMC algorithm parameters #################################################################################
    auto_stop        = True # Automatic stop using autocorrelation analysis
    burn_in          = 500 # (only works if auto_stop = False)
    write_iter       = 100 # check every N number of iterations
    write_thresh     = 100 # when to start writing parameters
    min_iter         = 1500 # minimum number of iterations before checking
    max_iter         = 10000 # max num ber of MCMC iterations before stopping
    #############################################################################################################
    # Final component fitting options
        # Note: by default, all options (except for fit_losvd) are set to 'True' for the initial fit, so 
        # as to make no assumptions on the presence of outflows
    fit_feii     = True # fit FeII (option for Type 2 AGN)
    fit_losvd    = False # fit LOSVD (Stellar population) in final model only
    fit_host     = True # fit host-galaxy using a template(used when LOSVD is false or total continuum S/N < 5)
    fit_power    = True # fit AGN power-law continuum (option for Type 2 AGN)
    fit_broad    = True # fit broad lines (option for Type 2 AGN)
    fit_narrow   = True # fit narrow lines (because why not)
    fit_outflows = True # fit outflows; DO NOT SET TO FALSE 
    tie_narrow   = True # tie all narrow components to [OIII]5007 (also ties outflow components)
    #############################################################################################################
    
    work_dir = spec_loc[i]+'/'
    run_dir,prev_dir = badass.setup_dirs(work_dir)
    file = glob.glob(work_dir+'*.fits')[0]
    # Uncomment below line to use previously-chosen templates
#     temp_dir = spec_loc[i]+'/'+'templates_final'

    print('\n Starting object %s...' % (spec_loc[i][-19:]) )
    start_time = time.time() # start timer

    # Determine fitting region
    fit_reg,good_frac = badass.determine_fit_reg(file,good_thresh,run_dir,fit_reg=fit_reg)
    if (fit_reg is None):
        print('\n Fit region too small! Moving to next object... \n')
        break
    elif (good_frac < good_thresh) and (fit_reg is not None): # if fraction of good pixels is less than good_threshold, then move to next object
        print('\n Not enough good channels above threshold! Moving onto next object... \n')
        break
    elif (good_frac >= good_thresh) and (fit_reg is not None):
        print('          Fitting region: (%d,%d)' % (fit_reg[0],fit_reg[1]))
        print('          Fraction of good channels = %0.2f' % (good_frac))
    
    # Prepare SDSS spectrum for fitting
    lam_gal,galaxy,templates,noise,velscale,vsyst,temp_list,z,ebv,npix,ntemp,temp_fft,npad = badass.sdss_prepare(file,fit_reg,temp_dir,run_dir,plot=True)
    print('          z = %0.4f' % z)
    print('          E(B-V) =  %0.4f' % ebv)
    print('          Velocity Scale = %0.4f (km/s/pixel) \n' % velscale)
    
    ###########################################################################################################
    
    # Testing for outflows 
    if (fit_outflows==True) & ((fit_reg[0]<=4400.)==True) & ((fit_reg[1] >=5800.)==True) & (mcbs_niter>0): # Only do this for Hb Region
        print('\n Running outflow tests...')
        fit_outflows = badass.outflow_test(lam_gal,galaxy,noise,run_dir,velscale,mcbs_niter)
        if fit_outflows==True:
            print('  Outflows detected: including outflow components in fit...')
        elif fit_outflows==False:
            print('  Outflows not detected: disabling outflow components from fit...')
    
    # Initialize maximum likelihood parameters
    param_dict = badass.initialize_mcmc(lam_gal,galaxy,run_dir,fit_reg=fit_reg,fit_type='init',
                                        fit_feii=fit_feii,fit_losvd=fit_losvd,fit_host=fit_host,
                                        fit_power=fit_power,fit_broad=fit_broad,
                                        fit_narrow=fit_narrow,fit_outflows=fit_outflows,
                                        tie_narrow=tie_narrow)

    # By generating the galaxy and FeII templates before, instead of generating them with each iteration, we save a lot of time
    gal_temp = badass.galaxy_template(lam_gal,age=None) # 'age=None' option selections a NLS1 template
    if (fit_feii==True):
        na_feii_temp,br_feii_temp = badass.initialize_feii(lam_gal,velscale,fit_reg)
    elif (fit_feii==False):
        na_feii_temp,br_feii_temp = None,None

    # Step 1: Test for Outflows by fitting the region 4750-5200 
    # Initialize function arguments
    # Peform maximum likelihood
    result_dict = badass.max_likelihood(param_dict,lam_gal,galaxy,noise,gal_temp,na_feii_temp,br_feii_temp,
                                   temp_list,temp_fft,npad,velscale,npix,vsyst,run_dir,monte_carlo=False)
    
    # If the total continuum level is < 5, disable the host-galaxy and losvd components 
    # (there is no point in fitting them because the fits will be horrible)
    if all(key in result_dict for key in ['gal_temp_amp','power_amp'])==True:
        if ((result_dict['gal_temp_amp']['res']+result_dict['power_amp']['res'])<5.):
            print('\n Total continuum level < 5.  Disabling host_galaxy/losvd components...\n')
            fit_host  = False
            fit_losvd = False

    # Initialize parameters for emcee
    param_dict = badass.initialize_mcmc(lam_gal,galaxy,run_dir,fit_reg=fit_reg,fit_type='final',
                                        fit_feii=fit_feii,fit_losvd=fit_losvd,fit_host=fit_host,
                                        fit_power=fit_power,fit_broad=fit_broad,
                                        fit_narrow=fit_narrow,fit_outflows=fit_outflows)
    
    # Replace initial conditions with best fit max. likelihood parameters (the old switcharoo)
    for key in result_dict:
        if key in param_dict:
            param_dict[key]['init']=result_dict[key]['res']
    print('\n     Final parameters:')
    for key in param_dict:
        print('          %s = %0.2f' % (key,param_dict[key]['init']) )
            
    #######################################################################################################
    # Run emcee
    print('\n Performing MCMC iterations...')
    # Extract relevant stuff from dicts
    param_names  = [param_dict[key]['name'] for key in param_dict ]
    init_params  = [param_dict[key]['init'] for key in param_dict ]
    bounds       = [param_dict[key]['plim'] for key in param_dict ]

    ndim, nwalkers = len(init_params), 50 # minimum walkers = 2*len(params)
    # initial parameters Maximum Likelihood best-fit
    pos = [init_params + 1e-2*np.random.randn(ndim) for i in range(nwalkers)]
    # Run emcee; writes progress to MCMC_chain.csv
    # args = arguments of lnprob (log-probability function)
    lnprob_args=(param_names,bounds,lam_gal,galaxy,noise,gal_temp,na_feii_temp,br_feii_temp,
          temp_list,temp_fft,npad,velscale,npix,vsyst,run_dir)
    
    sampler_chain,burn_in = badass.run_emcee(pos,ndim,nwalkers,run_dir,lnprob_args,init_params,param_names,
                                     auto_stop,burn_in,write_iter,write_thresh,min_iter,max_iter,threads=4)
    

    # Add chains to each parameter in param dictionary
    for i,key in enumerate(param_names):
        if key in param_dict:
            param_dict[key]['chain']=sampler_chain[:,:,i]
        
    print('\n Fitting chains... \n')
    param_dict = badass.param_plots(param_dict,burn_in,run_dir,save_plots=True)

    flux_dict  = badass.emline_flux_plots(burn_in,nwalkers,run_dir,save_plots=True)
    
    lum_dict   = badass.flux2lum(flux_dict,burn_in,nwalkers,z,run_dir,H0=71.0,Om0=0.27,save_plots=True)
    
    print('\n Saving Data... \n')
    # Write best fit parameters to fits table
    badass.write_param(param_dict,flux_dict,lum_dict,run_dir)
    # Write all chains to a fits table
    badass.write_chain(param_dict,flux_dict,lum_dict,run_dir)
    # Plot and save the best fit model and all sub-components
    badass.plot_best_model(param_dict,lam_gal,galaxy,noise,gal_temp,na_feii_temp,br_feii_temp,
                           temp_list,temp_fft,npad,velscale,npix,vsyst,run_dir)
    
    print('\n Cleaning up... \n')
    # Delete redundant files to cut down on space
    badass.cleanup(run_dir)
    
    elap_time = (time.time() - start_time)	   
    print("\n Total Runtime = %s" % (badass.time_convert(elap_time)))   
    print('\n Done! \n')
